# Bigquery anomaly detection with Gdelt

In this notebook we will query the Gdelt Global Entity Graph and find which entities are anomolus based on BQ ARIMA+ model

In [33]:
import json
#set params
BEGIN = "2021-03-01"
END = "2021-04-30"

HEIGHT = 400
WIDTH = 400

bq_params = {"BEGIN": BEGIN, "END": END}
bq_params = json.dumps(bq_params, indent=4, sort_keys=True, default=str)

### First step - get top entities by count, partitioned by day

In [164]:
%%bigquery --params {"BEGIN": "2021-03-01", "END": "2021-04-30"}

CREATE OR REPLACE TABLE cpg-cdp.trendspotting.gdelt_terms as (
WITH geg_data AS ((
    SELECT 
    groupId, 
    name, 
    a.entity as stringVal, 
    a.numMentions, 
    a.avgSalience, 
    eventTime, 
    polarity, 
    magnitude, 
    score 
    from (
    	SELECT polarity, 
        magnitude, 
        score, 
        FARM_FINGERPRINT(url) groupId, 
        CONCAT('Entity',entity.type) name, 
        FORMAT_TIMESTAMP("%Y-%m-%d", date, "UTC") eventTime, 
    	entity.mid mid FROM `gdelt-bq.gdeltv2.geg_gcnlapi`, UNNEST(entities) entity WHERE entity.mid is not null 
       and lang='en' and DATE(date) >= @BEGIN and DATE(date) <= @END
  ) b JOIN (
     SELECT APPROX_TOP_COUNT(entities.name, 1)[OFFSET(0)].value entity, #grab the entities from the nested json in the graph
        entities.mid mid, sum(entities.numMentions) as numMentions, 
        avg(entities.avgSalience) as avgSalience
      FROM `gdelt-bq.gdeltv2.geg_gcnlapi`, 
     unnest(entities) entities where entities.mid is not null and lang='en' 
     and DATE(date) >= @BEGIN and DATE(date) <= @END group by entities.mid
  ) a USING(mid)))
select * from 
(select *, RANK() OVER (PARTITION BY eventTime ORDER BY numMentions desc) as rank #Get ranks
from
(select stringVal, 
    name, 
    eventTime, 
    sum(numMentions) as numMentions, 
    avg(magnitude) as avgMagnitude 
    from geg_data group by 1,2,3) grouped_all) where rank < 300 #limit to top 500 entities partitioned by day - this is important and may want to report by different bands of popularity
)

Query complete after 0.00s: 100%|██████████| 13/13 [00:00<00:00, 8562.49query/s]                       


""


In [165]:
%%bigquery data
select * from cpg-cdp.trendspotting.gdelt_terms

Downloading: 100%|██████████| 18239/18239 [00:01<00:00, 15796.08rows/s]


In [166]:
data

,stringVal,name,eventTime,numMentions,avgMagnitude,rank
0,Oscar,EntityEVENT,2021-04-25,92651768,11.529545,114
1,Oscar,EntityEVENT,2021-04-23,39884852,8.786335,203
2,Oscar,EntityEVENT,2021-04-26,244635350,8.719139,76
3,Oscar,EntityEVENT,2021-04-11,22419746,5.824309,242
4,Oscar,EntityEVENT,2021-04-27,45582688,8.544565,194
...,...,...,...,...,...,...
18234,COVID-19,EntityCONSUMER_GOOD,2021-04-16,46620738,12.254167,197
18235,COVID-19,EntityCONSUMER_GOOD,2021-04-25,33056763,6.950000,193
18236,COVID-19,EntityCONSUMER_GOOD,2021-03-10,33793287,9.633333,235
18237,WhatsApp,EntityCONSUMER_GOOD,2021-04-30,22751793,19.737977,270


### Second step - run BQML on the data [link](https://cloud.google.com/blog/products/data-analytics/bigquery-ml-unsupervised-anomaly-detection)

In [167]:
%%bigquery

CREATE OR REPLACE MODEL trendspotting.gdelt_anomaly_v_1
OPTIONS(
  MODEL_TYPE='ARIMA_PLUS',
  TIME_SERIES_TIMESTAMP_COL='eventTime',
  TIME_SERIES_DATA_COL='numMentions',
  TIME_SERIES_ID_COL='stringVal',
  HOLIDAY_REGION='US' 
) AS
SELECT
  cast(eventTime as Date) as eventTime, stringVal, numMentions
FROM `cpg-cdp.trendspotting.gdelt_terms`

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1822.03query/s]                        


""


In [168]:
# now detect anomalies on the training set

In [169]:
%%bigquery anoms
SELECT
  * 
FROM
  ML.DETECT_ANOMALIES(MODEL `trendspotting.gdelt_anomaly_v_1`,
                      STRUCT(0.99 AS anomaly_prob_threshold)) a,
    `cpg-cdp.trendspotting.gdelt_terms` b where a.stringVal = b.stringVal and a.eventTime = CAST(b.eventTime  as TIMESTAMP)

Downloading: 100%|██████████| 17685/17685 [00:01<00:00, 16299.69rows/s]


In [170]:
anoms

,stringVal,eventTime,numMentions,is_anomaly,lower_bound,upper_bound,anomaly_probability,stringVal_1,name,eventTime_1,numMentions_1,avgMagnitude,rank
0,AP,2021-03-01 00:00:00+00:00,3.319966e+09,False,2.908441e+09,3.731491e+09,0.000000,AP,EntityORGANIZATION,2021-03-01,3319965958,13.160897,10
1,AP,2021-04-11 00:00:00+00:00,2.688041e+09,False,2.023297e+09,2.846347e+09,0.888976,AP,EntityORGANIZATION,2021-04-11,2688040809,12.225198,9
2,AP,2021-03-11 00:00:00+00:00,3.909541e+09,False,3.449758e+09,4.272809e+09,0.238575,AP,EntityORGANIZATION,2021-03-11,3909540753,12.323012,8
3,AP,2021-03-15 00:00:00+00:00,3.709317e+09,False,3.179298e+09,4.002349e+09,0.543814,AP,EntityORGANIZATION,2021-03-15,3709316794,14.953408,8
4,AP,2021-04-01 00:00:00+00:00,3.856016e+09,False,3.332082e+09,4.155133e+09,0.520455,AP,EntityORGANIZATION,2021-04-01,3856015801,12.627215,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17680,Centers for Disease Control and Prevention,2021-04-09 00:00:00+00:00,7.605644e+08,False,5.783087e+08,1.100858e+09,0.566081,Centers for Disease Control and Prevention,EntityORGANIZATION,2021-04-09,760564416,13.250726,37
17681,Centers for Disease Control and Prevention,2021-04-20 00:00:00+00:00,5.813854e+08,False,5.519929e+08,1.074542e+09,0.978352,Centers for Disease Control and Prevention,EntityORGANIZATION,2021-04-20,581385408,12.619479,42
17682,Centers for Disease Control and Prevention,2021-03-30 00:00:00+00:00,8.116683e+08,False,6.197556e+08,1.142305e+09,0.508073,Centers for Disease Control and Prevention,EntityORGANIZATION,2021-03-30,811668288,13.730859,34
17683,Centers for Disease Control and Prevention,2021-03-16 00:00:00+00:00,6.719213e+08,False,5.208997e+08,1.043449e+09,0.724687,Centers for Disease Control and Prevention,EntityORGANIZATION,2021-03-16,671921280,14.967183,37


In [171]:
anoms[anoms['stringVal'].str.contains("Ever Given")] #detected anomolies on Ever Given

,stringVal,eventTime,numMentions,is_anomaly,lower_bound,upper_bound,anomaly_probability,stringVal_1,name,eventTime_1,numMentions_1,avgMagnitude,rank
12053,Ever Given,2021-03-28 00:00:00+00:00,20261150.0,True,2.637634e+07,2.812184e+07,1.000000,Ever Given,EntityOTHER,2021-03-28,20261150,9.883846,248
12054,Ever Given,2021-03-29 00:00:00+00:00,35445880.0,False,3.434226e+07,3.608776e+07,0.506533,Ever Given,EntityOTHER,2021-03-29,35445880,9.474372,200
12055,Ever Given,2021-03-27 00:00:00+00:00,16565160.0,False,1.569241e+07,1.743791e+07,0.000000,Ever Given,EntityOTHER,2021-03-27,16565160,9.774194,287


In [172]:
## Look at anomaly charts

In [173]:
import altair as alt

def create_entity_count_charts_color(data, filter_terms, only_anom=False, category=None):
    
    if only_anom:
        filtered = anoms[data.is_anomaly == True]
        filtered = filtered[filtered.avgMagnitude > 9]
        filtered_terms = set(filtered.stringVal)
        data = data[data.stringVal.str.contains("|".join(filtered_terms))]
    
    chart_title = f"Anomalies by num mentions"
    if category is not None:
        data = data[data.name == category]
        chart_title = f"Anomalies by num mentions for {category}"
    
    
    if filter_terms is not None:
        data = data[data.stringVal.str.contains('|'.join(filter_terms))]
    data['eventTime'] = data['eventTime'].astype('datetime64[ns]')

    anomaly_chart = alt.Chart(data).mark_point().encode(
    x={"field": "eventTime", 'type': 'temporal', 'stack': None},
    y={'field': 'numMentions', 'type': 'quantitative', 'stack': None},
    tooltip=['eventTime','numMentions','is_anomaly', 'anomaly_probability', 'stringVal','avgMagnitude'],
    size='is_anomaly',
		).properties(
        height=HEIGHT,
        width=WIDTH*2
    ).interactive()
    
    line_chart = alt.Chart(data).mark_line().encode(
    x={"field": "eventTime", 'type': 'temporal', 'stack': None},
    y={'field': 'numMentions', 'type': 'quantitative', 'stack': None},
    tooltip=['eventTime','numMentions','is_anomaly', 'anomaly_probability', 'stringVal','avgMagnitude'],
    color='stringVal',
		).properties(
        title=chart_title,
        height=HEIGHT,
        width=WIDTH*2
    ).interactive()
    
    
    return alt.layer(anomaly_chart, line_chart)

alt.data_transformers.disable_max_rows()

create_entity_count_charts_color(anoms, filter_terms=['Ever Given','George Floyd'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


alt.LayerChart(...)

In [174]:
create_entity_count_charts_color(anoms, filter_terms=None, only_anom=True, category='EntityOTHER')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead


alt.LayerChart(...)

In [175]:
create_entity_count_charts_color(anoms, filter_terms=None, only_anom=True, category='EntityEVENT')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead


alt.LayerChart(...)

In [177]:
create_entity_count_charts_color(anoms, filter_terms=None, only_anom=True, category='EntityORGANIZATION')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead


alt.LayerChart(...)